In [24]:
import fasttext
import pandas as pd
import numpy as np
import matplotlib as mpl
import nltk,re,pprint
import sys,glob,os
from imp import reload
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'nltk'

In [2]:
kickstarter2016 = pd.read_csv("../datasets/kickstarterProjects/ks-projects-201612.csv", encoding="latin")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
 dataAnalyst = pd.read_csv("../datasets/DataAnalyst.csv", encoding="latin")

In [6]:
len(dataAnalyst["Industry"].unique())

89

In [7]:
dataAnalystSmall = dataAnalyst[['Job Description', 'Company Name', 'Industry']]

In [8]:
dataAnalystSmall

,Job Description,Company Name,Industry
0,Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,Social Assistance
1,Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,Health Care Services & Hospitals
2,Weâre looking for a Senior Data Analyst who ...,Squarespace\n3.4,Internet
3,Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,IT Services
4,ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,Sports & Recreation
...,...,...,...
2248,Maintains systems to protect data from unautho...,"Avacend, Inc.\n2.5",Staffing & Outsourcing
2249,Position:\nSenior Data Analyst (Corporate Audi...,Arrow Electronics\n2.9,Wholesale
2250,"Title: Technical Business Analyst (SQL, Data a...",Spiceorb,-1
2251,Summary\n\nResponsible for working cross-funct...,Contingent Network Services\n3.1,Enterprise Software & Network Solutions


In [9]:
dataAnalystGrouped = dataAnalystSmall.groupby(['Industry']).count()

In [10]:
pd.set_option('display.max_rows', 500)
print(dataAnalystGrouped.sort_values(by=['Job Description'], ascending=False))

                                           Job Description  Company Name
Industry                                                                
-1                                                     353           352
IT Services                                            325           325
Staffing & Outsourcing                                 323           323
Health Care Services & Hospitals                       151           151
Consulting                                             111           111
Computer Hardware & Software                           111           111
Investment Banking & Asset Management                   78            78
Enterprise Software & Network Solutions                 69            69
Internet                                                65            65
Banks & Credit Unions                                   51            51
Advertising & Marketing                                 51            51
Insurance Carriers                                 

In [17]:
for col in dataAnalystTopCatGrouped.columns: 
    print(col) 

Industry
Job Description


In [12]:
dataAnalystGrouped.shape

(89, 2)

In [13]:
dataAnalystTopCat = dataAnalystSmall[dataAnalystSmall["Industry"].isin(['IT Services', 'Staffing & Outsourcing', 'Health Care Services & Hospitals', 'Consulting',
'Computer Hardware & Software', 'Investment Banking & Asset Management', 'Enterprise Software & Network Solutions', 'Internet',
'Banks & Credit Unions', 'Advertising & Marketing'])]

In [14]:
dataAnalystTopCatGrouped = dataAnalystTopCat.groupby(['Industry'])['Job Description'].apply(lambda x: ','.join(x)).reset_index()

In [15]:
print(dataAnalystTopCatGrouped.shape)
dataAnalystTopCatGrouped.reset_index()

(10, 2)


,index,Industry,Job Description
0,0,Advertising & Marketing,About Known\n\nKnown is a modern marketing com...
1,1,Banks & Credit Unions,"Job Description:\n\nWithin GMC, Data & Portfol..."
2,2,Computer Hardware & Software,"If interested, please reach out to Iram Shah e..."
3,3,Consulting,About Us:\n\nNYSTEC is a non-profit technology...
4,4,Enterprise Software & Network Solutions,"Data Analyst Jersey City, NJ 12+ months contra..."
5,5,Health Care Services & Hospitals,Overview\n\nProvides analytical and technical ...
6,6,IT Services,Requisition NumberRR-0001939\nRemote:Yes\nWe c...
7,7,Internet,Weâre looking for a Senior Data Analyst who ...
8,8,Investment Banking & Asset Management,About Cubist\nCubist Systematic Strategies is ...
9,9,Staffing & Outsourcing,Title: Data Analyst\n\nPosition: 1+ Year Contr...


In [16]:
dataAnalystTopCatGrouped["Job Description"].shape

(10,)

In [23]:
for index, row in dataAnalystTopCatGrouped.iterrows():
    ofname = '../processFiles/trainCatFiles/cat' + str(index)+ '_' + '-'.join(row['Industry'].split()) + '.txt'
    print(row['Industry'],ofname)
    ofile = open(ofname,"a")
    ofile.writelines(row['Job Description'])

Advertising & Marketing ../processFiles/trainCatFiles/cat0_Advertising-&-Marketing.txt
Banks & Credit Unions ../processFiles/trainCatFiles/cat1_Banks-&-Credit-Unions.txt
Computer Hardware & Software ../processFiles/trainCatFiles/cat2_Computer-Hardware-&-Software.txt
Consulting ../processFiles/trainCatFiles/cat3_Consulting.txt
Enterprise Software & Network Solutions ../processFiles/trainCatFiles/cat4_Enterprise-Software-&-Network-Solutions.txt
Health Care Services & Hospitals ../processFiles/trainCatFiles/cat5_Health-Care-Services-&-Hospitals.txt
IT Services ../processFiles/trainCatFiles/cat6_IT-Services.txt
Internet ../processFiles/trainCatFiles/cat7_Internet.txt
Investment Banking & Asset Management ../processFiles/trainCatFiles/cat8_Investment-Banking-&-Asset-Management.txt
Staffing & Outsourcing ../processFiles/trainCatFiles/cat9_Staffing-&-Outsourcing.txt
